In [0]:
import os
# To work with Amazon S3 storage, set the following variables using
# your AWS Access Key and Secret Key
# Set the Region to where your files are stored in S3.
access_key = 'AKIA***AWHA'
secret_key = 'MSuJ***0mX/'
# Set the environment variables so boto3 can pick them up later.
# Do not modify the next three lines of code
os.environ['AWS_ACCESS_KEY_ID'] = access_key
os.environ['AWS_SECRET_ACCESS_KEY'] = secret_key
encoded_secret_key = secret_key.replace("/", "%2F").replace("+", "%2B")
# Set aws_region to where your S3 bucket was created
aws_region = "us-east-2"

In [0]:
from pyspark.sql import SparkSession

# Create a SparkConf and SparkContext
spark = SparkSession.builder.appName("amazon-reviews-project").getOrCreate()

# Update the Spark options to work with our AWS Credentials
# Do not modify the next three lines
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3." + aws_region + ".amazonaws.com")

In [0]:
file_path = 's3a://amazon-reviews-project-xh/landing/amazon_reviews_multilingual_US_v1_00.tsv'
sdf = spark.read.csv(file_path, sep='\t', header=True, inferSchema=True)
sdf

Out[177]: DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: date]

In [0]:
# Print schema
sdf.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [0]:
# Show dataframe
sdf.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   53096384| R63J84G1LOX6R|1563890119|     763187671|The Sandman Vol. ...|           Books|          4|            0|          1|   N|                N|ignore the review...|this is the first...| 1995-08-13|
|         US|   53096399|R1BALOA11Z06MT|1559947608|     381720534|The 22 Immutable ...|           Books|          4|    

In [0]:
# Show column names
sdf.columns

Out[180]: ['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [0]:
# Get the number of records in the dataframe
sdf.count()

Out[181]: 6931166

In [0]:
# Import some functions we will need later on
from pyspark.sql.functions import col, isnan, when, count, udf

# Set the Spark logging level to only show errors 
sc.setLogLevel("ERROR")

# Set up the path to an Amazon reviews data stored on S3 
bucket = 'amazon-reviews-project-xh/landing/'
filename = 'amazon_reviews_multilingual_US_v1_00.tsv' 
file_path = 's3a://' + bucket + filename

# Create a Spark Dataframe from the file on S3
sdf = spark.read.csv(file_path, sep='\t', header=True, inferSchema=True)

# Check the schema
sdf.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [0]:
# Take a sample of the data without replacement
sdf = sdf.sample(False, 0.25)

In [0]:
# Summary of dataframe
sdf.summary().show()

+-------+-----------+--------------------+--------------+--------------------+--------------------+--------------------------+----------------+------------------+------------------+-----------------+-------+-----------------+---------------+--------------------+
|summary|marketplace|         customer_id|     review_id|          product_id|      product_parent|             product_title|product_category|       star_rating|     helpful_votes|      total_votes|   vine|verified_purchase|review_headline|         review_body|
+-------+-----------+--------------------+--------------+--------------------+--------------------+--------------------------+----------------+------------------+------------------+-----------------+-------+-----------------+---------------+--------------------+
|  count|    1734340|             1734340|       1734340|             1734340|             1734340|                   1734340|         1734340|           1734340|           1734340|          1734340|1734340|    

In [0]:
# Look at statistics for specific columns
sdf.select("customer_id","product_id", "review_headline", "review_body", "star_rating", "helpful_votes", "total_votes").summary("count", "min", "max").show()

+-------+-----------+----------+--------------------+--------------------+-----------+-------------+-----------+
|summary|customer_id|product_id|     review_headline|         review_body|star_rating|helpful_votes|total_votes|
+-------+-----------+----------+--------------------+--------------------+-----------+-------------+-----------+
|  count|    1731997|   1731997|             1731991|             1731854|    1731997|      1731997|    1731997|
|    min|      10020|0001046314|              |Excelente disc...|          1|            0|          0|
|    max|   53096584|B01MXDEXCW|🙌🙌🙌🙌🙌🙌🙌🙌?...|🚼 🚺💮😁😁👕😃🔩...|          5|        10466|      12016|
+-------+-----------+----------+--------------------+--------------------+-----------+-------------+-----------+



In [0]:
# Check to see if the columns have NULL values 
sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in ["customer_id","product_id", "review_headline", "review_body", "star_rating", "helpful_votes", "total_votes"]] ).show()

+-----------+----------+---------------+-----------+-----------+-------------+-----------+
|customer_id|product_id|review_headline|review_body|star_rating|helpful_votes|total_votes|
+-----------+----------+---------------+-----------+-----------+-------------+-----------+
|          0|         0|             10|        143|          0|            0|          0|
+-----------+----------+---------------+-----------+-----------+-------------+-----------+



In [0]:
# Drop some of the records where the certain columns are empty (null or nan)
sdf = sdf.na.drop(subset=["customer_id","product_id", "review_headline", "review_body", "star_rating", "helpful_votes", "total_votes"])
sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in ["customer_id","product_id", "review_headline", "review_body", "star_rating", "helpful_votes", "total_votes"]] ).show()

+-----------+----------+---------------+-----------+-----------+-------------+-----------+
|customer_id|product_id|review_headline|review_body|star_rating|helpful_votes|total_votes|
+-----------+----------+---------------+-----------+-----------+-------------+-----------+
|          0|         0|              4|          0|          0|            0|          0|
+-----------+----------+---------------+-----------+-----------+-------------+-----------+



In [0]:
sdf.count()

Out[188]: 1731848

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def ascii_only(myvalue):
    if myvalue is not None:
        if isinstance(myvalue, str):
            return myvalue.encode('ascii', 'ignore').decode('ascii')
        elif isinstance(myvalue, int):
            return str(myvalue).encode('ascii', 'ignore').decode('ascii')
    return None

# Turn this function into a User-Defined Function (UDF)
ascii_udf = udf(ascii_only, StringType())

# Clean up the columns
sdf = sdf.withColumn("clean_customer_id", ascii_udf('customer_id')) 
sdf = sdf.withColumn("clean_product_id", ascii_udf('product_id')) 
sdf = sdf.withColumn("clean_review_headline", ascii_udf('review_headline')) 
sdf = sdf.withColumn("clean_review_body", ascii_udf('review_body'))
sdf = sdf.withColumn("clean_star_rating", ascii_udf('star_rating'))
sdf = sdf.withColumn("clean_helpful_votes", ascii_udf('helpful_votes'))
sdf = sdf.withColumn("clean_total_votes", ascii_udf('total_votes'))

# Re-check the cleaned columns
sdf.select("clean_customer_id", "clean_product_id", "clean_review_headline", "clean_review_body", "clean_star_rating", "clean_helpful_votes", "clean_total_votes").summary("count", "min", "max").show()


+-------+-----------------+----------------+---------------------+--------------------+-----------------+-------------------+-----------------+
|summary|clean_customer_id|clean_product_id|clean_review_headline|   clean_review_body|clean_star_rating|clean_helpful_votes|clean_total_votes|
+-------+-----------------+----------------+---------------------+--------------------+-----------------+-------------------+-----------------+
|  count|          1731848|         1731848|              1731848|             1731848|          1731848|            1731848|          1731848|
|    min|           100000|      0001046314|                     |                    |                1|                  0|                0|
|    max|          9999998|      B01MXDEXCW| ~~~~~tedious trip...|~~~~~~~~~~~~~~~~~...|                5|                999|              996|
+-------+-----------------+----------------+---------------------+--------------------+-----------------+-------------------+-----------

In [0]:
output_file_path="s3://amazon-reviews-project-xh/raw/clean_amazon_reviews_multilingual_US_v1_00.tsv" 
sdf.write.mode("overwrite").options(header='True', delimiter='\t').csv(output_file_path)

In [0]:
# Read the TSV file back into a DataFrame
sdf = spark.read.option("header", "true").option("delimiter", "\t").csv(output_file_path)
